In [247]:
import geopandas
import pandas as pd

In [248]:
path_geo='../../data/geodata_bh/BAIRRO_POPULAR.shp'

In [249]:
shp_file = geopandas.read_file(path_geo)
shp_file=shp_file.to_crs(epsg=4326)
shp_file.to_file('../../data/geodata_bh/BAIRRO_POPULAR_4326.geojson', driver='GeoJSON')

In [250]:
# shp_file.NOME=shp_file.NOME.apply(lambda x: x+'_BH')

In [251]:
df=pd.read_csv('../../data/data2/processed/steps_2.2.csv',encoding='latin-1' )

In [252]:
gdf_deliveries = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df["lng"], df["lat"]))

In [253]:
dfgeo = geopandas.sjoin(gdf_deliveries, shp_file, how='left', op='within')

/home/caloja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_27719/1300309330.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  dfgeo = geopandas.sjoin(gdf_deliveries, shp_file, how='left', op='within')


In [254]:
df_geo_not_in_belohorizonte=dfgeo[dfgeo['NOME'].isnull()].drop(columns=['index_right','ID', 'CODIGO', 'NOME', 'AREA_KM2', 'PERIMETR_M'])
df_geo_in_belohorizonte=dfgeo[~dfgeo['NOME'].isnull()].drop(columns=['index_right','ID', 'CODIGO', 'AREA_KM2', 'PERIMETR_M'])

In [255]:
df_mg = geopandas.read_file('../../data/data2/MG.json')

In [273]:
df_warehouse=pd.read_csv('../../data/data2/warehouses.csv',encoding='latin-1' )
geo_warehouse = geopandas.GeoDataFrame(df_warehouse, geometry=geopandas.points_from_xy(df_warehouse["lng"], df_warehouse["lat"]))

In [274]:
merge_ciu_ware = geopandas.sjoin(df_mg, geo_warehouse, how='right', op='contains')

/home/caloja/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_27719/3770579690.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  merge_ciu_ware = geopandas.sjoin(df_mg, geo_warehouse, how='right', op='contains')


In [277]:
merge_ciu_ware.NOME.value_counts()

Belo Horizonte    46
Contagem          13
Betim              2
Sete Lagoas        1
Ipatinga           1
Name: NOME, dtype: int64

In [281]:
df.groupby(['lat','lng']).agg({'delivery_id':'count'}).sort_values(by='delivery_id',ascending=False)

delivery_id
lat      lng                  
-19.9618 -44.0285       239564
-19.9211 -43.9155         9951
-19.9524 -43.9388         8996
-19.8916 -44.0520         6313
-19.8586 -43.9591         4717
...                        ...
-19.9251 -43.9605            1
         -43.9637            1
         -43.9679            1
         -43.9702            1
-19.9128 -44.0732            1

[105929 rows x 1 columns]

In [282]:
geo_warehouse

,id,name,id_company,location,lat,lng,geometry
0,1947,TAG BH,1205,"{""address"":""R. Vicentina de Souza, 230 - Sagra...",-19.9064,-43.9187,POINT (-43.91870 -19.90640)
1,3391,Shippify Warehouse BH,1,"{""address"":""R. José Maria de Lacerda, 1900 - C...",-19.9618,-44.0285,POINT (-44.02850 -19.96180)
2,3616,Fedex WH,3437,"{""address"":""FedEx Transportadora, R. Sagitário...",-19.9703,-44.0676,POINT (-44.06760 -19.97030)
3,3670,CAFÉ ITAOCA,657,"{""address"":""R. Ulisses Marcondes Escobar, 36 -...",-19.9714,-43.9731,POINT (-43.97310 -19.97140)
4,3692,Em Casa,657,"{""address"":""R. Safira, 451 - Prado, Belo Horiz...",-19.9266,-43.9636,POINT (-43.96360 -19.92660)
...,...,...,...,...,...,...,...
58,6217,BH warehouse,9447,"{""address"":""R. José Maria de Lacerda, 1900 - C...",-19.9618,-44.0285,POINT (-44.02850 -19.96180)
59,6257,SETE LAGOAS,9386,"{""address"":""R. Lassance Cunha, 205 - Centro, S...",-19.4595,-44.2430,POINT (-44.24300 -19.45950)
60,6259,IPATINGA,9386,"{""address"":""R. Vinte e Oito de Abril, 227 - Ce...",-19.4782,-42.5257,POINT (-42.52570 -19.47820)
61,6264,CONTAGEM,9386,"{""address"":""Av. João César de Oliveira, 2794 -...",-19.9384,-44.0488,POINT (-44.04880 -19.93840)


In [256]:
# gdf_deliveries_mg_out = geopandas.sjoin(df_geo_not_in_belohorizonte, df_mg, how='left', op='within')

In [257]:
# gdf_deliveries_mg_out_si=gdf_deliveries_mg_out[~gdf_deliveries_mg_out.NOME.isnull()].drop(columns=['index_right','GEOCODIGO', 'UF'])
# gdf_deliveries_mg_out_no=gdf_deliveries_mg_out[gdf_deliveries_mg_out.NOME.isnull()]

In [258]:
# df_complete=pd.concat([df_geo_in_belohorizonte,gdf_deliveries_mg_out_si])

In [259]:
# df_complete.shape

In [260]:
# df_complete=df_complete[~df_complete.delivery_id.isin(gdf_deliveries_mg_out_no.delivery_id.unique())]

In [261]:
df_complete=df_geo_in_belohorizonte[~df_geo_in_belohorizonte.delivery_id.isin(df_geo_not_in_belohorizonte.delivery_id.unique())]

In [262]:
# df_complete.to_csv('../../data/data2/processed/steps_2.3_withPoint.csv',index=False)

In [263]:
group_id=df_complete.groupby('delivery_id')
ids_unique=df_complete.delivery_id.unique()

In [264]:
names=df_complete.NOME.unique()

In [265]:
group_id.get_group(ids_unique[6]).iloc[0]['action']

'pickup'

In [266]:
tuples=[]
for i in ids_unique:
    delivery=group_id.get_group(i).iloc[0]
    p_nome=''
    d_nome=''
    if delivery['action']=='pickup':
        p_nome=delivery['NOME']
        delivery_2=group_id.get_group(i).iloc[1]
        d_nome=delivery_2['NOME']
    else:
        d_nome=delivery['NOME']
        delivery_2=group_id.get_group(i).iloc[1]
        p_nome=delivery_2['NOME']

    tuples.append((p_nome,d_nome))

In [267]:
import networkx as nx

In [268]:
G=nx.DiGraph()

In [269]:
G.add_nodes_from(names)

In [270]:
G.add_edges_from(tuples)

In [271]:
nx.write_gexf(G, "../../data/data2/processed/steps_graph_barrios.gexf")